In [1]:
from keras.utils import np_utils
import numpy as np
import h5py
np.random.seed(10)

C:\Users\Stephanie\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
f=h5py.File("data_RGB_train_last_0_17_shuffle.h5","r")

In [3]:
data=f['feature+label']

In [4]:
data.shape

(38880, 100, 100, 3)

In [5]:
y=data[:,0,0,0]

In [6]:
y.shape

(38880,)

In [7]:
y_TrainOneHot = np_utils.to_categorical(y)

In [8]:
y_TrainOneHot.shape

(38880, 18)

In [9]:
x_train=np.array(data)

In [10]:
x_train.shape

(38880, 100, 100, 3)

In [11]:
x_train[:,0,0,0]=0

In [12]:
f.close()

In [13]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D

In [14]:
model = Sequential()

In [15]:
model.add(Conv2D(filters=20,
                 kernel_size=(4,4),
                 padding='same',
                 input_shape=(100,100,3),
                 activation='relu'))

In [16]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [17]:
model.add(Conv2D(filters=40,
                 kernel_size=(4,4),
                 padding='same',
                 activation='relu'))

In [18]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [19]:
model.add(Dropout(0.25))

In [20]:
model.add(Flatten())

In [21]:
model.add(Dense(128, activation='relu'))

In [22]:
model.add(Dropout(0.5))

In [23]:
model.add(Dense(18,activation='softmax'))

In [24]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 100, 100, 20)      980       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 40)        12840     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 40)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 25, 40)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3200128   
__________

In [25]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',metrics=['accuracy']) 

In [26]:
train_history=model.fit(x=x_train,
                        y=y_TrainOneHot,validation_split=0.2,
                        epochs=11, batch_size=150,verbose=2)

Train on 31104 samples, validate on 7776 samples
Epoch 1/11
 - 535s - loss: 1.5027 - acc: 0.4665 - val_loss: 0.7866 - val_acc: 0.7433
Epoch 2/11
 - 534s - loss: 0.8552 - acc: 0.6822 - val_loss: 0.5876 - val_acc: 0.7845
Epoch 3/11
 - 523s - loss: 0.6676 - acc: 0.7495 - val_loss: 0.4699 - val_acc: 0.8327
Epoch 4/11
 - 522s - loss: 0.5569 - acc: 0.7900 - val_loss: 0.4076 - val_acc: 0.8495
Epoch 5/11
 - 524s - loss: 0.4864 - acc: 0.8130 - val_loss: 0.3038 - val_acc: 0.8984
Epoch 6/11
 - 527s - loss: 0.4268 - acc: 0.8341 - val_loss: 0.2493 - val_acc: 0.9088
Epoch 7/11
 - 524s - loss: 0.3753 - acc: 0.8534 - val_loss: 0.2646 - val_acc: 0.8951
Epoch 8/11
 - 529s - loss: 0.3373 - acc: 0.8699 - val_loss: 0.1912 - val_acc: 0.9261
Epoch 9/11
 - 526s - loss: 0.3274 - acc: 0.8747 - val_loss: 0.2217 - val_acc: 0.9113
Epoch 10/11
 - 525s - loss: 0.2835 - acc: 0.8901 - val_loss: 0.1427 - val_acc: 0.9460
Epoch 11/11
 - 524s - loss: 0.2620 - acc: 0.8973 - val_loss: 0.1332 - val_acc: 0.9478


In [27]:
model.save('model_052201_RGB.h5')

In [28]:
del x_train,y_TrainOneHot,y

In [29]:
f2=h5py.File("data_RGB_test_last_0_17_shuffle.h5","r")

In [30]:
test_data=f2['feature+label']

In [31]:
test_data.shape

(19440, 100, 100, 3)

In [32]:
y_test=test_data[:,0,0,0]

In [33]:
y_test.shape

(19440,)

In [34]:
y_TestOneHot = np_utils.to_categorical(y_test)

In [35]:
x_test=np.array(test_data)

In [36]:
x_test.shape

(19440, 100, 100, 3)

In [37]:
x_test[:,0,0,0]=0

In [38]:
f2.close()

In [39]:
scores = model.evaluate(x_test , y_TestOneHot)

19440/19440 [==============================] - 138s 7ms/step


In [40]:
scores[1]

0.8405864197530865

In [41]:
prediction=model.predict_classes(x_test)

In [42]:
import pandas as pd
pd.crosstab(y_test,prediction,rownames=['label'],colnames=['predict'])

predict,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
label,,,,,,,,,,,,,,,,,,
0.0,899,0,60,0,0,0,0,0,0,0,0,85,36,0,0,0,0,0
1.0,0,920,72,0,9,6,0,0,0,0,0,0,0,4,48,21,0,0
2.0,38,0,956,0,0,0,0,0,10,0,0,36,0,40,0,0,0,0
3.0,0,5,0,1009,0,0,0,31,1,0,0,0,0,28,6,0,0,0
4.0,0,0,0,0,665,134,146,0,0,0,0,0,0,0,11,84,0,40
5.0,0,0,0,0,190,810,60,0,0,1,19,0,0,0,0,0,0,0
6.0,0,0,0,0,6,85,884,0,0,80,25,0,0,0,0,0,0,0
7.0,0,0,0,0,26,19,0,854,126,4,0,0,0,35,0,16,0,0
8.0,0,0,0,0,0,34,0,36,1009,0,0,0,0,1,0,0,0,0
